<a href="https://colab.research.google.com/github/arnavsacheti/OC-VAE/blob/main/OC_FaceDect_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!pip install -q cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp39-cp39-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.350 requires google-api-python-client>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [3]:
!pip install -q lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# imports typing inferences
from typing import Any, Optional, Tuple

# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

In [5]:
import os

from torch import optim, nn, utils, Tensor, Size
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchmetrics.functional import accuracy
from torchsummary import summary
import lightning as L

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from PIL import Image

In [6]:
# Import Kaggle.json for dataset download
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iamdestegosaurus","key":"0ed4c8660f01a0cbe92270fd267596b6"}'}

In [7]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d xhlulu/140k-real-and-fake-faces
!unzip -q -n 140k-real-and-fake-faces.zip -d dataset

-rw-r--r-- 1 root root 72 Apr 27 16:08 kaggle.json
100% 3.75G/3.75G [00:43<00:00, 99.6MB/s]
100% 3.75G/3.75G [00:43<00:00, 92.5MB/s]


In [8]:
!head dataset/train.csv

,original_path,id,label,label_str,path
0,/kaggle/input/flickrfaceshq-dataset-nvidia-part-7/images1024x1024-20191222T221133Z-035/images1024x1024/31000/31355.png,31355,1,real,train/real/31355.jpg
1,/kaggle/input/flickrfaceshq-dataset-nvidia-part-1/images1024x1024-20191222T221133Z-004/images1024x1024/02000/02884.png,02884,1,real,train/real/02884.jpg
2,/kaggle/input/flickrfaceshq-dataset-nvidia-part-4/images1024x1024-20191222T221133Z-019/images1024x1024/33000/33988.png,33988,1,real,train/real/33988.jpg
3,/kaggle/input/flickrfaceshq-dataset-nvidia-part-1/images1024x1024-20191222T221133Z-003/images1024x1024/53000/53875.png,53875,1,real,train/real/53875.jpg
4,/kaggle/input/flickrfaceshq-dataset-nvidia-part-6/images1024x1024-20191222T221133Z-030/images1024x1024/24000/24149.png,24149,1,real,train/real/24149.jpg
5,/kaggle/input/flickrfaceshq-dataset-nvidia-part-2/images1024x1024-20191222T221133Z-008/images1024x1024/30000/30627.png,30627,1,real,train/real/30627.jpg
6,/kaggle/input/flickrfaceshq-d

In [9]:
BATCH_SIZE = 64
MAX_EPOCHS = 100
DATASET_PATH = "dataset"
IMAGE_PATH = "dataset/real_vs_fake/real-vs-fake"

In [10]:
class FakeRealDataset(L.LightningDataModule):
    """
    A custom dataset class for handling face images labeled as fake or real. 
    Inherits from the PyTorch Dataset class.
    """

    def __init__(
        self,
        dataset: pd.DataFrame,
        transform: Optional[transforms.Compose] = None,
        target_transform: Optional[transforms.Compose] = None,
    ) -> None:
        """
        Initialize the FakeRealDataset class with the given dataset and optional transforms.

        :param dataset: Dataset containing the image paths and labels in a pandas DataFrame.
        :param transform: Optional image transformations, passed as a torchvision.transforms.Compose object.
        :param target_transform: Optional target transformations, passed as a torchvision.transforms.Compose object.
        """
        self.image_dataset = dataset
        self.classes = self.image_dataset.label_str.unique()
        self.encode = {k: i for i, k in enumerate(self.classes)}
        
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) -> int:
        """
        Get the length of the dataset.

        :return: The number of samples in the dataset.
        """
        return len(self.image_dataset)

    def __getitem__(self, idx: int) -> Tuple[Any, int]:
        """
        Get an item from the dataset using its index.

        :param idx: The index of the desired sample in the dataset.
        :return: A tuple containing the image and its label.
        """
        img_path = self.image_dataset.iloc[idx, 0]
        label = self.encode[self.image_dataset.iloc[idx, 1]]

        image = Image.open(os.path.join(IMAGE_PATH, img_path))

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label


In [11]:
class ModifiedVariationalAutoEncoder(L.LightningModule):
    def __init__(self):
        super(ModifiedVariationalAutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(inplace=True),
        )

        self.mean = nn.Linear(1000, 100)
        self.logvar = nn.Linear(1000, 100)

      
        self.decoder = nn.Sequential(
            nn.Linear(100, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 32 * 32 * 32),
            nn.BatchNorm1d(32 * 32 * 32),
            nn.ReLU(inplace=True),
            nn.Unflatten(1, Size([32, 32, 32])),
            nn.ConvTranspose2d(32, 64, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=1, padding=1, bias=False),
        )

    def reparameterize(self, mean, logvar):
        std = 0.5 * torch.exp(logvar)
        z = (std.data.new(std.size()).normal_()) * std + mean
        return z

    def forward(self, x):
        enocde = self.encoder(x)

        mean = self.mean(enocde)
        logvar = self.logvar(enocde)
        z = self.reparameterize(mean, logvar)

        x_hat = self.decoder(z)
        return x_hat, mean, logvar
    
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x_hat, mean, logvar = self(x)

        loss = self.loss_function(x, x_hat, mean, logvar)
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x_hat, mean, logvar = self(x)
        
        loss = self.loss_function(x, x_hat, mean, logvar)
        self.log("val_loss", loss, sync_dist=True, prog_bar=True)
    #     # self.log("val_acc", acc, prog_bar=True)
        
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def loss_function(self, x, x_hat, mean, logvar):
        mse_loss = nn.MSELoss(reduction="sum")
        reconstruction_loss = mse_loss(x_hat, x)
        kld_loss = -0.5 * torch.sum(1+logvar-torch.exp(logvar)-mean**2)
        return reconstruction_loss + kld_loss

In [12]:
preprocess = transforms.Compose(
  [
    # transforms.Resize(256),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ]
)

In [13]:
# Define input and output parameters for the dataset
X_params = ["path"]
y_params = ["label_str"]

# Load and balance the dataset
train = pd.read_csv(os.path.join(DATASET_PATH, "train.csv"))
train = train[X_params + y_params]
train_real = FakeRealDataset(dataset=train[train.label_str=="real"], transform=preprocess)
train_fake = FakeRealDataset(dataset=train[train.label_str=="fake"], transform=preprocess)

valid = pd.read_csv(os.path.join(DATASET_PATH, "valid.csv"))
valid = valid[X_params + y_params]
valid_real = FakeRealDataset(dataset=valid[valid.label_str=="real"], transform=preprocess)
valid_fake = FakeRealDataset(dataset=valid[valid.label_str=="fake"], transform=preprocess)

test = pd.read_csv(os.path.join(DATASET_PATH, "test.csv"))
test = test[X_params + y_params]
test_real = FakeRealDataset(dataset=test[test.label_str=="real"], transform=preprocess)
test_fake = FakeRealDataset(dataset=test[test.label_str=="fake"], transform=preprocess)

In [14]:
train_real_loader = DataLoader(dataset=train_real, batch_size=BATCH_SIZE, shuffle=True)
valid_real_loader = DataLoader(dataset=valid_real, batch_size=BATCH_SIZE, shuffle=False)
test_real_loader = DataLoader(dataset=test_real, batch_size=BATCH_SIZE, shuffle=False)


In [15]:
!mkdir /content/lightning_logs

In [ ]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(log_every_n_steps=10, max_epochs=MAX_EPOCHS)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: True, using: 8 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: True, using: 8 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=autoencoder, train_dataloaders=train_real_loader, val_dataloaders=valid_real_loader)

Exception in device=TPU:0: INTERNAL: From /job:tpu_worker/replica:0/task:0:
2 root error(s) found.
  (0) INTERNAL: stream did not block host until done; was already in an error state
	 [[{{node XRTExecute}}]]
	 [[XRTExecute_G12]]
  (1) INTERNAL: stream did not block host until done; was already in an error state
	 [[{{node XRTExecute}}]]
0 successful operations.
0 derived errors ignored.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 334, in _mp_start_fn
    _start_fn(index, pf_cfg, fn, args)
  File "/usr/local/lib/python3.9/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 328, in _start_fn
    fn(gindex, *args)
  File "/usr/local/lib/python3.9/dist-packages/lightning/pytorch/strategies/launchers/xla.py", line 107, in _wrapping_function
    results = function(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/lightning/pytorch/trainer/trainer.py", line 559, in _fit_impl


ProcessExitedException: ignored